# Nuisance Parameters
In this notebook, Bayesian Optimisation is used on a fundamentally 1D problem, pretending to be 2D with the addition of a nuisance parameter.

by using an 'automatic relevance detection' (ARD) kernel, the problems caused by nuisance parameters can be mitigated, by using 'less precision' along those dimensions. Below, the predictions from ARD and non-ARD surrogates is plotted. See how the ARD surrogate matches the true objective function much better.

An ARD kernel is formed when different length scales are used for each dimension

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

In [ ]:
# Make deterministic
np.random.seed(100)

# Target Function

In [ ]:
xmin, xmax = -2, 10
nuisance_min, nuisance_max = 0, 10
xs = np.linspace(xmin, xmax, 100)
nuisance = np.linspace(nuisance_min, nuisance_max, 100)

f = lambda x: np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1) + \
    np.random.normal(0, 0.02, size=None if isinstance(x, float) else x.shape)
f2D = lambda x, nuisance: f(x)

ys = f(xs)
best_y = np.max(ys)

X,Y = np.meshgrid(xs, nuisance)
Z = f2D(X, Y)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xs, ys)
plt.show()

In [ ]:
tp.surface_3D(X, Y, Z)

In [ ]:
bounds1D = [('x', xmin, xmax)]
bounds2D = [('x', xmin, xmax), ('nuisance', nuisance_min, nuisance_max)]

gp_params = dict(
    alpha = 1e-5, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.Matern(nu=2.5) + gp.kernels.WhiteKernel(),
    n_restarts_optimizer = 3,
)

op1D = tb.Optimiser(f, 'max', bounds1D, pre_phase_trials=4, settings_preset='default')
op1D.surrogate_factory =  tm.SciKitGPSurrogate.Factory(gp_params)

rec1D = tp.PlottingRecorder(op1D)

op2D = tb.Optimiser(f2D, 'max', bounds2D, pre_phase_trials=4, settings_preset='default')
op2D.surrogate_factory =  tm.SciKitGPSurrogate.Factory(gp_params=gp_params)
rec2D = tp.PlottingRecorder(op2D)


gp_params_ARD = dict(
    alpha = 1e-5, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.Matern(length_scale=(1.0, 1.0), nu=2.5) + gp.kernels.WhiteKernel(),
    n_restarts_optimizer = 3,
)
op2DARD = tb.Optimiser(f2D, 'max', bounds2D, pre_phase_trials=4, settings_preset='default')
op2DARD.surrogate_factory =  tm.SciKitGPSurrogate.Factory(gp_params=gp_params_ARD)
rec2DARD = tp.PlottingRecorder(op2DARD)

In [ ]:
tg.OptimiserProgressBar(op1D)
op1D.run(max_trials=25)

tg.OptimiserProgressBar(op2D)
np.random.seed(0)
op2D.run(max_trials=25)

tg.OptimiserProgressBar(op2DARD)
np.random.seed(0)
op2DARD.run(max_trials=25)

In [ ]:
tp.plot_error_over_time(rec1D, true_best=best_y);
tp.plot_error_over_time(rec2D, true_best=best_y);
tp.plot_error_over_time(rec2DARD, true_best=best_y);

In [ ]:
tp.plot_trial_1D(rec1D, param='x', trial_num=15, true_objective=f);

In [ ]:
tp.interactive_plot_trial_2D(rec2D, trial_num=15, true_objective=f2D);

In [ ]:
tp.interactive_plot_trial_2D(rec2DARD, trial_num=15, true_objective=f2D);